# Set-up of the Cloud App using Azure CLI

These Azure CLI commands can be used to set-up and update the web app.

In [ ]:
import subprocess
import json
import os

import psycopg2

## Login in to Azure CLI

In [ ]:
# This command will bring up a browser window in which to login.
subprocess.run(["az", "login"])

## Create a resource group

This contains all the resources the app needs.

In [ ]:
RESOURCE_GROUP_NAME = "xxxxxxxxxxx"
RESOURCE_GROUP_LOCATION = "UK south"

In [ ]:
# Create a resource group.
subprocess.run([
    "az", "group", "create",
    "--name", RESOURCE_GROUP_NAME,
    "--location", RESOURCE_GROUP_LOCATION
])

## Create storage for static files

The app's static files (html and javascript).

In [ ]:
STORAGE_ACCOUNT_NAME = "xxxxxx"
STATIC_CONTAINER_NAME = "xxxxxxxx"
MEDIA_CONTAINER_NAME = "xxxxxxxxx"

In [ ]:
subprocess.run([
    "az", "storage", "account", "create",
    "--name", STORAGE_ACCOUNT_NAME,
    "--resource-group", RESOURCE_GROUP_NAME,
    "--location", RESOURCE_GROUP_LOCATION,
    "--allow-blob-public-access", "true"
])

In [ ]:
# Static storage
subprocess.run([
    "az", "storage", "container", "create",
    "--account-name", STORAGE_ACCOUNT_NAME,
    "--account-key", "xxxxxxxxxxxxxxxxxx",
    "--name", STATIC_CONTAINER_NAME,
    "--public-access", "blob",
])

In [ ]:
# Media storage
subprocess.run([
    "az", "storage", "container", "create",
    "--account-name", STORAGE_ACCOUNT_NAME,
    "--account-key", "xxxxxxxxxxxxxxxxxx",
    "--name", MEDIA_CONTAINER_NAME,
    "--public-access", "blob",
])

## Create the PostgreSQL database

https://learn.microsoft.com/en-gb/azure/postgresql/flexible-server/quickstart-create-server-portal

https://learn.microsoft.com/en-us/cli/azure/postgres/flexible-server


In [ ]:
POSTGRES_SERVER_NAME = "xxxxxx"
POSTGRES_SERVER_USERNAME = "xxxxxxx"
POSTGRES_SERVER_PASSWORD = "xxxxxxxx"

In [ ]:
# Create the server.
# This takes a long time.
# Having a problem with this (wants a value for --private-dns-zone) -> try doing this through the portal
subprocess.run([
    "az", "postgres", "flexible-server", "create",
    "--name", POSTGRES_SERVER_NAME,
    "--resource-group", RESOURCE_GROUP_NAME,
    "--location", RESOURCE_GROUP_LOCATION,
    "--public-access", "all",
    "--admin-user", POSTGRES_SERVER_USERNAME,
    "--admin-password", POSTGRES_SERVER_PASSWORD,
    "--sku-name", "Standard_B1ms",
    "--tier", "Burstable",
    #"--private-dns-zone", "blabla",
    #"--yes"
])

In [ ]:
# Allow the web app access to the database
subprocess.run([
    "az", "postgres", "flexible-server", "firewall-rule", "create",
    "--resource-group", RESOURCE_GROUP_NAME,
    "--name", POSTGRES_SERVER_NAME,
    "--rule-name", "AllowAllWindowsAzureIps",
    "--start-ip-address", "0.0.0.0",
    "--end-ip-address", "0.0.0.0",
])

## Create the web app

In [ ]:
WEB_APP_SERVICE_PLAN_NAME = "xxxxxxxx"
WEB_APP_NAME = "xxxxxxxx"  # needs to be unique

In [ ]:
# Web app service plan
subprocess.run([
    "az", "appservice", "plan", "create",
    "--name", WEB_APP_SERVICE_PLAN_NAME,
    "--resource-group", RESOURCE_GROUP_NAME,
    "--sku", "B1",
    "--is-linux"
])

In [ ]:
# Create the web-app
subprocess.run([
    "az", "webapp", "create",
    "--name", WEB_APP_NAME,
    "--plan", WEB_APP_SERVICE_PLAN_NAME,
    "--resource-group", RESOURCE_GROUP_NAME,
    "--runtime", "PYTHON|3.10",
    "--deployment-local-git"
])

Enable health check. This allows the removal of 'unhealthy instances' of the app that will sometimes cause the app to not be able to start.

https://learn.microsoft.com/en-us/azure/app-service/monitor-instances-health-check?tabs=dotnet

web app > monitoring > health check

#### Or ........

Simple web app installation

Set up app
From:
https://learn.microsoft.com/en-us/azure/app-service/tutorial-python-postgresql-app?tabs=django%2Cwindows&pivots=azure-portal#1-create-app-service-and-postgresql

In the azure portal search for "web app database" and select the "web app + database" option


Local Git deployment
From:
https://learn.microsoft.com/en-us/azure/app-service/deploy-local-git?tabs=portal

1) web app > deployment centre > setting > local git, save

2) deployment center > settings > git clone uri. Add to pycharm project's remote.


pushes have to be from master


## Set the app's environmental variables

A single environmental variable, `DJANGO_SETTINGS` is a JSON (encoded as a string) that contains:
- Values used for the app's settings.
- Values used for connecting to databases.

#### Local (development)

In [ ]:
environ_settings_local = {
    # Standard.
    # https://docs.djangoproject.com/en/5.1/ref/settings/#static-files
    "DEBUG": "True",
    "ALLOWED_HOSTS": ["127.0.0.1"],
    "CSRF_TRUSTED_ORIGINS": [],
    "SECRET_KEY": "xxxxxxxxxxxx",
    "DATABASES": {
        "default": {
            "ENGINE": "django.db.backends.sqlite3",
            "NAME": "xxxxxx/xxxxx/db.sqlite3",
            "HOST": "",
            "PORT": "",
            "USER": "",
            "PASSWORD": ""
        },
    },
    "STATIC_URL": "/static/",
    "MEDIA_URL": "/media/",
    "STORAGES": {
        "default": {
            "BACKEND": "django.core.files.storage.FileSystemStorage",
        },
        "staticfiles": {
            "BACKEND": "django.contrib.staticfiles.storage.StaticFilesStorage"
        }
    },
}

In [ ]:
# Paste this string into PyCharm's run configuration window for the run django ("runserver"),
# as the environmental setting "DJANGO_SETTINGS"
json.dumps(environ_settings_local)

#### Azure (production)

In [ ]:
environ_settings_production = {
    # Standard Django settings.
    # https://docs.djangoproject.com/en/5.1/ref/settings/#static-files
    "DEBUG": "True",
    "ALLOWED_HOSTS": [f"{WEB_APP_NAME}.azurewebsites.net"],
    "CSRF_TRUSTED_ORIGINS": [f"https://{WEB_APP_NAME}.azurewebsites.net"],
    "SECRET_KEY": "xxxxxxxxxx",
    "DATABASES": {
        "default": {
            "ENGINE": "django.db.backends.postgresql",
            "NAME": "postgres",
            "HOST": f"{POSTGRES_SERVER_NAME}.postgres.database.azure.com",
            "PORT": "",
            "USER": POSTGRES_SERVER_USERNAME,
            "PASSWORD": POSTGRES_SERVER_PASSWORD
        },
    },
    "STATIC_URL": f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{STATIC_CONTAINER_NAME}/",
    "MEDIA_URL": f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{MEDIA_CONTAINER_NAME}/",
    "STORAGES": {
        "default": {
            "BACKEND": "sigweb.backend.AzureMediaStorage",
        },
        "staticfiles": {
            "BACKEND": "sigweb.backend.AzureStaticStorage"
        }
    },
    
}

In [ ]:
pestr = json.dumps(environ_settings_production)
subprocess.run([
    "az", "webapp", "config", "appsettings", "set",
    "--resource-group", RESOURCE_GROUP_NAME,
    "--name", WEB_APP_NAME,
    "--settings",
    f"DJANGO_SETTINGS={pestr}"
])

#### Other Azure variables

https://learn.microsoft.com/en-us/azure/app-service/reference-app-settings?tabs=kudu%2Cdotnet

In [ ]:
subprocess.run([
    "az", "webapp", "config", "appsettings", "set",
    "--resource-group", RESOURCE_GROUP_NAME,
    "--name", WEB_APP_NAME,
    "--settings",
    "WEBSITE_START_TIME_LIMIT=500"
])

In [ ]:
subprocess.run([
    "az", "webapp", "config", "appsettings", "set",
    "--resource-group", RESOURCE_GROUP_NAME,
    "--name", WEB_APP_NAME,
    "--settings",
    "WEBSITES_CONTAINER_START_TIME_LIMIT=800"
])

## Git

Push the local source code of the Web App up to the app deployed on Azure.

1. Set up the deployed app's remote origin on PyCharm.
    * Get URL, user and password from Web App > deployment centre > local git/FTPA credentials.
    * In Pycharm > Git > manage remotes
2. Push the code to the deployed app's remote.


## Migrate

web app > development tools > SSH

Must do the first two of these everytime.

```
python manage.py makemigrations

python manage.py migrate

python manage.py collectstatic

python manage.py createsuperuser
```

## Go to website

It may take a while to get a good connection

In [ ]:
f"https://{WEB_APP_NAME}.azurewebsites.net/admin"

gunicorn --bind=0.0.0.0 --timeout 600 --workers=4 sigweb.wsgi:application

## Remote connection to the web app's database

If you want to connect to the web app's SQL database from your computer, your computer's public IP will need to be added to the firewall rules of the app.

https://www.psycopg.org/docs/usage.html

In [ ]:
# Get the "public IP" address of this computer.
# Use the returned value below.
subprocess.run(["curl", "ifcfg.me"])

In [ ]:
# Allow local access to the database
# May take a while.
my_local_ip_address = "46.17.164.101"
subprocess.run([
    "az", "postgres", "flexible-server", "firewall-rule", "create",
    "--resource-group", RESOURCE_GROUP_NAME,
    "--name", POSTGRES_SERVER_NAME,
    "--rule-name", "AllowMyIP",
    "--start-ip-address", my_local_ip_address,
    "--end-ip-address", my_local_ip_address,
])

In [ ]:
# Connect to the database.
cd = dict(
    host=f"{POSTGRES_SERVER_NAME}.postgres.database.azure.com",
    port="5432",
    dbname="postgres",
    user=POSTGRES_SERVER_USERNAME,
    password=POSTGRES_SERVER_PASSWORD,
    sslmode='require'
)
conn = psycopg2.connect(**cd)

In [ ]:
cur = conn.cursor()
cur.execute("""
SELECT * FROM xxxxxxx
""")
print(cur.fetchall())

In [ ]:
conn.close()